### 2.7.1 Pure fluid solutions
Whilst the variables required for each pure-fluid solubility model are different, we take a very similar approach to calculating fluid properties for each. In general pure-fluid models provide analytical expressions for the concentration of a volatile species dissolved in a magma as a function of either its partial-pressure or fugacity in the vapour phase. To provide a homogeneous framework, each model must have a fugacity model associated with it. Where a solubility model uses partial-pressure in place of fugacity, the fugacity model is set to being an ideal gas.

In general, an analytical expression does not exist for the inverse calculation, i.e., calculating the saturation pressure given the concentration of dissolved volatile species. The calculation must then be solved numerically. Numerical methods obtain the answer by making guesses for the saturation pressure, checking whether the predicted concentration of the dissolved volatile species matches the measured value, and refining the guesses as appropriate. When the saturation pressure is found, this equality holds:

\begin{equation}
 V^L_\textrm{measured} - V^L_\textrm{predicted}(P,...) = 0
\end{equation}

When the guessed pressure is too high the left hand side of the equation will be negative, and if the guessed pressure is too low it will be positive, We therefore require an algorithm to find the root of the equation. To this end we employ the scipy.optimize.root_scalar() method. The advantage of using a generic scipy algorithm, in preference to creating a bespoke routine, is that it affords us flexibility in how the problem is solved, and the algorithm is underlain by robust, well-tested, efficient code.

### 2.7.2 Finding saturation pressure of mixed-fluid
Describe how scipy.optimize is used here

### 2.7.3 Finding the equilibrium fluid composition
When calculating the dissolved volatile concentrations in a progressively degassing magma, we break the calculation into steps, each with a small decrement in pressure. As pressure decreases the mass of fluid in equilibrium with the magma increases and its composition will change. At each stage we must, therefore, solve for the mass and composition of the fluid in equilibrium with the magma, given the total mass of volatiles in the system. This calculation is implemented in MixedFluid.calculate_equilibrium_fluid_comp(), which in turn is called by MixedFluid.calculate_degassing_path(). Here we summarise the methodology for the calculation.

Mass balance in the system can be expressed in terms of the mole fractions of H$_2$O ($X_{\textrm{H}_2 \textrm{O}}$) and CO$_2$ ($X_{\textrm{CO}_2}$) in the silicate-liquid ($L$), the H$_2$O-CO$_2$O fluid ($F$), and the system total ($T$), and the fraction of fluid present ($f$).

\begin{equation}
X^F_{\textrm{CO}_2} + X^F_{\textrm{H}_2 \textrm{O}} = 1
\end{equation}

\begin{equation}
(1-f)X^L_{\textrm{CO}_2} + f X^F_{\textrm{CO}_2} = X^T_{\textrm{CO}_2}
\end{equation}

\begin{equation}
(1-f)X^L_{\textrm{H}_2 \textrm{O}} + f X^F_{\textrm{H}_2 \textrm{O}} = X^T_{\textrm{H}_2 \textrm{O}}
\end{equation}

The fraction of fluid ($f$) can be eliminated by substitution, obtaining Equation --NUMBER-- of --CITATION--:

\begin{equation}
\frac{X^T_{\textrm{CO}_2}-X^L_{\textrm{CO}_2}}{X^F_{\textrm{CO}_2}-X^L_{\textrm{CO}_2}} - \frac{X^T_{\textrm{H}_2\textrm{O}}-X^L_{\textrm{H}_2\textrm{O}}}{X^F_{\textrm{H}_2\textrm{O}}-X^L_{\textrm{H}_2\textrm{O}}} = 0
\end{equation}

In principle, this equation allows us to numerically solve for mass balance; however, the equation is valid only for both $X^F_{\textrm{CO}_2} \neq X^L_{\textrm{CO}_2}$ and $X^F_{\textrm{H}_2 \textrm{O}} \neq X^L_{\textrm{H}_2 \textrm{O}}$. When $X^F_{\textrm{CO}_2} \rightarrow X^L_{\textrm{CO}_2}$ or $X^F_{\textrm{H}_2 \textrm{O}} \rightarrow X^L_{\textrm{H}_2 \textrm{O}}$ the equation tends to $\pm \infty$, which the scipy.optimize.root_scalar() routine will mistake for a root of the equation. The regions over which the expression is not infinite can be found; however we find a more reliable approach when using the scipy.optimize library is to use the equation in a different form.

Rearranging the CO$_2$ mass-balance expression allows the fraction of fluid ($f$) to be calculated:

\begin{equation}
f = \frac{X^T_{\textrm{CO}_2}-X^L_{\textrm{CO}_2}}{X^F_{\textrm{CO}_2}-X^L_{\textrm{CO}_2}}
\end{equation}

This is valid for $X^F_{\textrm{CO}_2} \neq X^L_{\textrm{CO}_2}$. It is much more unlikely that a solution will exist near $X^F_{\textrm{CO}_2} = X^L_{\textrm{CO}_2}$ than near $X^F_{\textrm{H}_2 \textrm{O}} = X^L_{\textrm{H}_2 \textrm{O}}$. By not directly eliminating $f$ by dividing through by both ($X^F_{\textrm{CO}_2} - X^L_{\textrm{CO}_2}$) and ($X^F_{\textrm{H}_2 \textrm{O}} - X^L_{\textrm{H}_2 \textrm{O}}$)  we significantly increase the reliability of using this expression with scipy.optimize.root_scalar(). In this expression, the only known value is $X^T_{\textrm{CO}_2}$. However, if a guess is provided for $X^F_{\textrm{CO}_2}$, the partial pressure of CO$_2$ can be calculated and the corresponding value for $X^L_{\textrm{CO}_2}$ obtained. In general the guess for $X^F_{\textrm{CO}_2}$ will not satisfy mass balance when the same parameters are applied to both CO$_2$ and H$_2$O. 

Since $X^F_{\textrm{H}_2 \textrm{O}} = 1 - X^F_{\textrm{CO}_2}$, when we provide a guess for $X^F_{\textrm{CO}_2}$ we also fix $X^F_{\textrm{H}_2 \textrm{O}}$. This in turn fixes the partial pressure of H$_2$O, allowing us to calculate $X^L_{\textrm{H}_2 \textrm{O}}$ consistent with that guess. We can then test whether our guess for $X^F_{\textrm{CO}_2}$ satisfies mass balance by checking for the equality:

\begin{equation}
(1-f)X^L_{\textrm{H}_2 \textrm{O}} + f X^F_{\textrm{H}_2 \textrm{O}} - X^T_{\textrm{H}_2 \textrm{O}} = 0
\end{equation}

The value of $X^F_{\textrm{CO}_2}$ that uniquely satisfies mass balance can then be found numerically by solving this equation using scipy.optimize.root_scalar().